In [ ]:
!pip install sahi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=37e757d2fc7575de9caf830db82ed464711eefaae5aade92243c1e525562e982
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.8.0.76
    Uninstalling opencv-python-4.8.0.76:
      Successfully uninstalled opencv-python-4.8.0.76


In [ ]:
from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation
from sahi.utils.file import save_json
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [ ]:
data_path = "/content/drive/MyDrive/Thesis_Organized/Data/Ground Truth/annotations.csv"
df = pd.read_csv(data_path).drop(columns = ["Unnamed: 0"])
df.head()

,id,name,track,z_order,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,folder,bbox_x,bbox_y,bbox_width,bbox_height,segmentation
0,0,L1Ap5_1-1-109.png,Track1,0,411.65,248.47,312.63,149.45,223.32,36.45,Train,209.48,254.64,217.19,239.21,"[217.19, 27.01, 215.91, 28.29, 214.62, 29.58, ..."
1,0,L1Ap5_1-1-109.png,Track2,0,384.92,256.98,339.97,263.66,406.79,249.08,Train,327.71,237.92,93.82,43.73,"[399.68, 239.21, 398.4, 240.5, 397.11, 240.5, ..."
2,1,L1Ap5_1-1-142.png,Track1,0,567.18,283.10,495.49,275.20,440.15,264.27,Val,427.96,218.63,151.65,48.87,"[435.67, 253.36, 434.38, 254.64, 433.1, 254.64..."
3,1,L1Ap5_1-1-142.png,Track2,0,316.88,223.56,243.37,181.04,130.37,128.18,Val,116.95,279.08,214.62,124.75,"[127.23, 117.03, 125.95, 118.32, 124.66, 118.3..."
4,1,L1Ap5_1-1-142.png,Track3,0,364.27,251.51,356.98,269.74,347.26,291.61,Val,332.86,210.92,46.27,66.88,"[355.99, 243.07, 354.7, 244.35, 354.7, 245.64,..."


In [ ]:
df.isna().sum()

id              0
name            0
track           0
z_order         0
p1_x            0
p1_y            0
p2_x            0
p2_y            0
p3_x            0
p3_y            0
folder          0
bbox_x          0
bbox_y          0
bbox_width      0
bbox_height     0
segmentation    0
dtype: int64

In [ ]:
df.segmentation[0]

'[217.19, 27.01, 215.91, 28.29, 214.62, 29.58, 214.62, 30.87, 213.34, 32.15, 213.34, 33.44, 213.34, 34.72, 213.34, 36.01, 213.34, 37.3, 213.34, 38.58, 213.34, 39.87, 213.34, 41.15, 213.34, 42.44, 214.62, 43.73, 215.91, 45.01, 215.91, 46.3, 215.91, 47.58, 217.19, 47.58, 218.48, 48.87, 218.48, 50.16, 219.76, 51.44, 221.05, 52.73, 221.05, 54.01, 222.33, 55.3, 223.62, 56.59, 224.9, 57.87, 224.9, 59.16, 226.19, 60.45, 226.19, 61.73, 227.47, 61.73, 228.76, 63.02, 228.76, 64.3, 230.04, 65.59, 231.33, 66.88, 231.33, 68.16, 232.61, 69.45, 233.9, 70.73, 235.18, 72.02, 235.18, 73.31, 236.47, 74.59, 237.75, 75.88, 239.04, 77.16, 239.04, 78.45, 240.32, 79.74, 241.61, 81.02, 242.89, 82.31, 242.89, 83.59, 244.18, 84.88, 245.46, 86.17, 246.75, 87.45, 248.04, 88.74, 248.04, 90.02, 249.32, 91.31, 250.61, 92.6, 251.89, 93.88, 251.89, 95.17, 253.18, 96.46, 254.46, 97.74, 255.75, 99.03, 257.03, 100.31, 257.03, 101.6, 258.32, 102.89, 259.6, 104.17, 260.89, 105.46, 262.17, 106.74, 262.17, 108.03, 263.46, 109

In [ ]:
def correct_segmentations(seg):
  corrected_seg = []
  for i in seg[1:-1].split(","):
    corrected_seg.append(float(i))
  return corrected_seg

In [ ]:
coco_train = Coco()
coco_test = Coco()
coco_val = Coco()

coco_train.add_category(CocoCategory(id=0, name='Track'))
coco_test.add_category(CocoCategory(id=0, name='Track'))
coco_val.add_category(CocoCategory(id=0, name='Track'))

main_dir = "/content/drive/MyDrive/Thesis_Organized/Data/Images_cropped"
ids = np.unique(df.id.values)
for i in tqdm(ids):
  temp = df[df.id == i].copy().reset_index()
  img_path = os.path.join(main_dir, temp.folder.values[0], temp.name.values[0])
  coco_image = CocoImage(file_name=temp.name.values[0], height=517, width=658)
  for j in temp.index:
    segs = correct_segmentations(temp.segmentation.iloc[j])
    coco_image.add_annotation(
      CocoAnnotation(
      # bbox=temp.loc[j, ["bbox_x", "bbox_y", "bbox_width", "bbox_height"]].values.tolist(),   # the function will automatically assign bounding boxes more accurately
      category_id=0,
      category_name='Track',
      segmentation = [segs]
      )
    )
  if temp.folder[j] == "Train":
      coco_train.add_image(coco_image)
  if temp.folder[j] == "Test":
      coco_test.add_image(coco_image)
  if temp.folder[j] == "Val":
      coco_val.add_image(coco_image)

save_json(data=coco_train.json, save_path="/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_train_data.json")
save_json(data=coco_test.json, save_path="/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_test_data.json")
save_json(data=coco_val.json, save_path="/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_val_data.json")

100%|██████████| 986/986 [00:04<00:00, 204.78it/s]


In [ ]:
coco_train.stats

{'num_images': 791,
 'num_annotations': 2402,
 'num_categories': 1,
 'num_negative_images': 0,
 'num_images_per_category': {'Track': 791},
 'num_annotations_per_category': {'Track': 2402},
 'min_num_annotations_in_image': 1,
 'max_num_annotations_in_image': 10,
 'avg_num_annotations_in_image': 3.0366624525916563,
 'min_annotation_area': 479,
 'max_annotation_area': 12425,
 'avg_annotation_area': 4854.815986677769,
 'min_annotation_area_per_category': {'Track': 479},
 'max_annotation_area_per_category': {'Track': 12425}}

In [ ]:
coco_test.stats

{'num_images': 98,
 'num_annotations': 286,
 'num_categories': 1,
 'num_negative_images': 0,
 'num_images_per_category': {'Track': 98},
 'num_annotations_per_category': {'Track': 286},
 'min_num_annotations_in_image': 1,
 'max_num_annotations_in_image': 7,
 'avg_num_annotations_in_image': 2.9183673469387754,
 'min_annotation_area': 480,
 'max_annotation_area': 11504,
 'avg_annotation_area': 4858.849650349651,
 'min_annotation_area_per_category': {'Track': 480},
 'max_annotation_area_per_category': {'Track': 11504}}

In [ ]:
coco_val.stats

{'num_images': 97,
 'num_annotations': 309,
 'num_categories': 1,
 'num_negative_images': 0,
 'num_images_per_category': {'Track': 97},
 'num_annotations_per_category': {'Track': 309},
 'min_num_annotations_in_image': 1,
 'max_num_annotations_in_image': 9,
 'avg_num_annotations_in_image': 3.185567010309278,
 'min_annotation_area': 657,
 'max_annotation_area': 11736,
 'avg_annotation_area': 4815.569579288026,
 'min_annotation_area_per_category': {'Track': 657},
 'max_annotation_area_per_category': {'Track': 11736}}

# Check if saved correctly

In [ ]:
import json

with open('/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_train_data.json', 'r') as f:
  data_train = json.load(f)

data_train.keys()

dict_keys(['images', 'annotations', 'categories'])

In [ ]:
data_train["images"][0]

{'height': 517, 'width': 658, 'id': 1, 'file_name': 'L1Ap5_1-1-109.png'}

In [ ]:
data_train["categories"][0]

{'id': 0, 'name': 'Track', 'supercategory': 'Track'}

In [ ]:
data_train["annotations"][0].keys()

dict_keys(['iscrowd', 'image_id', 'bbox', 'segmentation', 'category_id', 'id', 'area'])

In [ ]:
id_to_name = {}
for i in data_train["images"]:
  image_id = i["id"]
  image_name = i["file_name"]
  id_to_name[image_id] = image_name

In [ ]:
for image in data_train["images"]:
  name = image["file_name"]
  temp = np.unique(df[df.name == name].folder.values)[0]
  if temp == "Train":
    continue
  else:
    print(name)

In [ ]:
count = 0
id = 1
for annotation in data_train["annotations"]:
  if annotation["image_id"] != id:
    count = 0
    id = annotation["image_id"]
  image_name = id_to_name[annotation["image_id"]]
  true = df.loc[df.name == image_name, ["segmentation"]].values[count][0]
  true = correct_segmentations(true)
  true = np.array(true, dtype = int)
  pred = annotation["segmentation"][0]
  pred = np.array(pred, dtype = int)
  if (pred == true).all():
    count = count + 1
    continue
  else:
    print(pred, true)
    break

All segmentations and image names are correct

In [4]:
import os

def print_tree(startpath, depth=1, level=0):
    prefix = "    " * level
    print(prefix + os.path.basename(startpath) + "/")
    if level < depth:
        for item in os.listdir(startpath):
            path = os.path.join(startpath, item)
            if os.path.isdir(path):
                print_tree(path, depth, level+1)
            else:
                print(prefix + "    " + item)

# Specify the directory and depth
directory = "/content/drive/MyDrive/Thesis_Organized/Data"  # Current directory
specified_depth = 2  # Adjust this to your desired depth

print_tree(directory, specified_depth)

Data/
    Original Images/
        Test/
        Val/
        Train/
        annotations.xml
    Ground Truth/
        Val/
        Test/
        Train/
        annotations.csv
    Images_cropped/
        Train/
        Test/
        Val/
    COCO_Format/
        Visualization_tools.ipynb
        coco_train_data.json
        coco_test_data.json
        coco_val_data.json
    1. Dataset_exploration.ipynb
    2. processing the images.ipynb
    3. transform_data_to_coco.ipynb
